In [15]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from textwrap import dedent
from langchain.output_parsers import OutputFixingParser
from langchain.schema import OutputParserException
from langchain_openai import ChatOpenAI
from langfuse.callback import CallbackHandler
import settings

In [18]:
# langfuse_handler = CallbackHandler(
#     secret_key=settings.LANGFUSE_SECRET_KEY,
#     public_key=settings.LANGFUSE_PUBLIC_KEY,
#     host=settings.LANGFUSE_HOST, 
# )

# def parse_response(output, parser, llm=ChatOpenAI(model="gpt-4o", openai_api_key=settings.OPENAI_API_KEY)):
#     try:
#         parsed = parser.parse(output)
#     except OutputParserException:
#         try:
#             new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
#             parsed = new_parser.parse(output.content)
#             print("Fixed parsing errors.")
#         except Exception as e:
#             print("Failed to fix parsing errors.")
#     return parsed

# def llm_invoke(llm, messages):
#     return llm.invoke(messages.to_messages(), {"callbacks": [langfuse_handler]})

# class ResponseModel(BaseModel):
#     soft_skills: List[str] = Field(
#         description="List of soft skills keywords found in the job description"
#     )
    
#     hard_skills: List[str] = Field(
#         description="List of hard skills keywords found in the job description"
#     )
    
#     other_skills: Optional[List[str]] = Field(
#         description="List of other keywords, which are not categorized as hard skills or soft skills, but are essential for the job"
#     )

# def get_overall_keywords(text, llm):
#     parser = PydanticOutputParser(pydantic_object=ResponseModel)
#     format_instructions = parser.get_format_instructions()
#     prompt_template = dedent(
#         """
#         You are an AI assistant specialized in .....

#         Format your response as follows:
#         {format_instructions}

#         If no keywords fit a particular category, return an empty list for that category. Your goal is to extract accurate and concise keywords that improve ATS matching.

#         Text:
#         {text}"""
#     )

#     prompt = ChatPromptTemplate(
#         messages=[HumanMessagePromptTemplate.from_template(prompt_template)],
#         input_variables=["job_description", "format_instructions"],
#     )

#     _input = prompt.format_prompt(
#         text=text, format_instructions=format_instructions
#     )
#     # output = llm.invoke(_input.to_messages())
#     output = llm_invoke(llm, _input)

#     parsed = parse_response(output.content, parser)

#     return parsed

In [19]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain.schema import OutputParserException
from langchain_openai import ChatOpenAI
from langfuse.callback import CallbackHandler
import settings

In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain.schema import OutputParserException
from langchain_openai import ChatOpenAI
from langfuse.callback import CallbackHandler
import mysql.connector
import settings
from pydantic import BaseModel, Field

# Configuration for handling callbacks and LLM setup
langfuse_handler = CallbackHandler(
    secret_key=settings.LANGFUSE_SECRET_KEY,
    public_key=settings.LANGFUSE_PUBLIC_KEY,
    host=settings.LANGFUSE_HOST, 
)

def parse_response(output, parser, llm=ChatOpenAI(model="gpt-4o", openai_api_key=settings.OPENAI_API_KEY)):
    try:
        parsed = parser.parse(output)
    except OutputParserException:
        try:
            new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
            parsed = new_parser.parse(output.content)
            print("Fixed parsing errors.")
        except Exception as e:
            print("Failed to fix parsing errors.")
    return parsed

def llm_invoke(llm, messages):
    return llm.invoke(messages.to_messages(), {"callbacks": [langfuse_handler]})

class SQLQueryModel(BaseModel):
    sql_query: str = Field(
        description="SQL query generated based on the user's natural language input."
    )

def generate_sql_query(user_input, llm):
    """
    This function generates a SQL query based on the user's input using GPT-4 and the relationships in the cricket database.
    """
    # Define the parser for SQL output
    parser = PydanticOutputParser(pydantic_object=SQLQueryModel)
    format_instructions = parser.get_format_instructions()

    # Template for prompting the LLM to generate SQL queries
    prompt_template = dedent(
        """
        You are an AI assistant specialized in generating SQL queries from natural language.

        The database consists of the following tables and relationships:
        - `batting`: Contains information about players' batting performance.
          - Columns: `Runs` (runs scored), `Mins` (minutes batted), `BF` (balls faced), `4s`, `6s`, `SR` (strike rate), `Pos` (batting position), `Dismissal` (how the batsman got out), `Inns` (1 for 1st innings, 2 for 2nd innings), `Opposition`, `Ground`, `Start Date`, `t20_match_no`, `player_id`, `ground_id`, `opposition_country_id`, `home_away`, `match_result_id`, `in_finals_id`.
          - Foreign Keys: `player_id` references `player(player_id)`, `ground_id` references `ground(ground_id)`,
            `opposition_country_id` references `country(country_id)`, `match_result_id` references `match_result(match_result_id)`,
            `in_finals_id` references `in_finals(match_stage_id)`, `t20_match_no` to identify specific matches.
        - `bowling`: Contains information about players' bowling performance.
          - Columns: `Overs` (number of overs bowled), `Mdns` (number of maidens bowled), `Runs` (runs conceded), `Wkts` (wickets taken), `Econ` (economy rate), `Pos` (bowling position), `Inns` (1 for 1st innings, 2 for 2nd innings), `t20_match_no`, `player_id`, `match_result_id`, `home_away`, `in_finals_id`, `start_date`.
          - Foreign Keys: `player_id` references `player(player_id)`, `ground_id` references `ground(ground_id)`,
            `opposition_country_id` references `country(country_id)`, `match_result_id` references `match_result(match_result_id)`,
            `in_finals_id` references `in_finals(match_stage_id)`, `t20_match_no` to identify specific matches.
        - `fielding`: Contains information about players' fielding performance.
          - Columns: `Dis` (number of dismissals), `Ct` (catches taken), `St` (stumpings), `Ct Wk` (catches as wicketkeeper), `Ct Fi` (catches as a fielder), `Inns` (1 for 1st innings, 2 for 2nd innings), `t20_match_no`, `player_id`, `match_result_id`, `home_away`, `start_date`.
          - Foreign Keys: `player_id` references `player(player_id)`, `ground_id` references `ground(ground_id)`,
            `opposition_country_id` references `country(country_id)`, `match_result_id` references `match_result(match_result_id)`,
            `in_finals_id` references `in_finals(match_stage_id)`, `t20_match_no` to identify specific matches.
        - `player`: Stores details about players (`player_id` as primary key).
          - Columns: `player_id`, `player_name`, `country_id` (references `country(country_id)` to identify which country the player belongs to).
        - `country`: Stores information about countries (`country_id` as primary key).
          - Columns: `country_id`, `country_name` (e.g., 'India', 'Australia').
        - `ground`: Stores information about match grounds (`ground_id` as primary key).
        - `match_result`: Stores match results (`match_result_id` as primary key).
          - Columns: `match_result_id`, `result` (e.g., 'Won', 'Lost', 'Draw', 'No Result'), `opposition_country_id` (references `country(country_id)`).
        - `in_finals`: Stores information on whether a match was a final (`match_stage_id` as primary key).

        When generating the SQL query, consider the following:
        - Ensure appropriate joins between tables based on foreign key relationships.
        - Handle multiple conditions, such as filtering by runs, wickets, and specific years.
        - Make sure to include specific match details by using `t20_match_no` where applicable.
        - Use Common Table Expressions (CTEs) or subqueries if needed to calculate aggregates or percentages.
        - Use `NOT IN` subqueries to filter out matches involving certain players when requested.
        - Return only the columns that are necessary to answer the user's question.
        - If you encounter a player name or any entity that is misspelled, use fuzzy matching or approximate search techniques to identify the closest match.
        - To determine match outcomes involving a specific player (e.g., Arshdeep Singh), join `player` and `match_result` to verify the country and match outcome.
        - Use the `home_away` column from `batting`, `bowling`, or `fielding` to determine if the match was played at home, away, or neutral.
        - Use `t20_match_no` as a unique identifier for each match, linking batting, bowling, and fielding records.
        - When determining the number of matches played, consider using joins across multiple tables (`batting`, `bowling`, etc.) and apply `UNION` if needed to avoid double counting.
        - Ensure that the conditions for finals, such as checking `in_finals_id`, are correctly applied to identify final matches.
        - Include additional details like `start_date` and `opposition_country_id` to provide more context for match results, such as the opponent team and match date.
        - When calculating performance metrics (e.g., economy rate, win percentage when conditions are met), use CTEs or subqueries to first identify relevant matches and then apply aggregate functions to derive the final output.

        Format your response as follows:
        {format_instructions}

        Given the user's question, generate an appropriate SQL query that joins the necessary tables based on the foreign key relationships.

        User Input:
        {user_input}
        """
    )

    prompt = ChatPromptTemplate(
        messages=[HumanMessagePromptTemplate.from_template(prompt_template)],
        input_variables=["user_input", "format_instructions"],
    )

    # Format the prompt with user input and format instructions
    _input = prompt.format_prompt(
        user_input=user_input, format_instructions=format_instructions
    )
    
    # Invoke the LLM to generate the SQL query
    output = llm_invoke(llm, _input)
    
    # Parse the output to extract the SQL query
    parsed = parse_response(output.content, parser)
    sql_query = parsed.sql_query

    # Execute the SQL query against MySQL database
    try:
        connection = mysql.connector.connect(
            host=settings.DB_HOST,
            user=settings.DB_USER,
            password=settings.DB_PASSWORD,
            database=settings.DB_NAME,
            port=settings.DB_PORT
        )
        cursor = connection.cursor()
        cursor.execute(sql_query)
        result = cursor.fetchall()
        for row in result:
            print(row)
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

    return result

In [ ]:
if __name__ == "__main__":
    user_question = "What is the win percentage of India when Arshdeep took 3 wickets?"
    llm = ChatOpenAI(model="gpt-4o", openai_api_key=settings.OPENAI_API_KEY)
    result = generate_sql_query(user_question, llm)
    print("Query Result:", result)

In [16]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain.schema import OutputParserException
from langchain_openai import ChatOpenAI
from langfuse.callback import CallbackHandler
import mysql.connector
# import pymysql as mysql_connector
import settings
from pydantic import BaseModel, Field


# Configuration for handling callbacks and LLM setup
langfuse_handler = CallbackHandler(
    secret_key=settings.LANGFUSE_SECRET_KEY,
    public_key=settings.LANGFUSE_PUBLIC_KEY,
    host=settings.LANGFUSE_HOST, 
)

def parse_response(output, parser, llm=ChatOpenAI(model="gpt-4o", openai_api_key=settings.OPENAI_API_KEY)):
    try:
        parsed = parser.parse(output)
    except OutputParserException:
        try:
            new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
            parsed = new_parser.parse(output.content)
            print("Fixed parsing errors.")
        except Exception as e:
            print("Failed to fix parsing errors.")
    return parsed

def llm_invoke(llm, messages):
    return llm.invoke(messages.to_messages(), {"callbacks": [langfuse_handler]})

class SQLQueryModel(BaseModel):
    sql_query: str = Field(
        description="SQL query generated based on the user's natural language input."
    )

def execute_sql_query(sql_query):
    """
    This function executes a given SQL query against the MySQL database and returns the result.
    """
    # Execute the SQL query against MySQL database
    try:
        connection = mysql.connector.connect(
            host=settings.DB_HOST,
            user=settings.DB_USER,
            password=settings.DB_PASSWORD,
            database=settings.DB_NAME,
            port=settings.DB_PORT
        )

        cursor = connection.cursor()
        cursor.execute(sql_query)
        result = cursor.fetchall()
        for row in result:
            print(row)
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

    return result


if __name__ == "__main__":

    test_sql_query = """
    WITH matches_with_3_wickets AS (
        SELECT 
            bo.t20_match_no
        FROM 
            bowling bo
        JOIN 
            player p ON bo.player_id = p.player_id
        WHERE 
            p.player_name = 'Arshdeep Singh'
            AND bo.Wkts = 2  -- Took exactly 2 wickets
    ),
    total_matches AS (
        SELECT COUNT(DISTINCT t20_match_no) AS total
        FROM matches_with_3_wickets
    ),
    won_matches AS (
        SELECT COUNT(DISTINCT bo.t20_match_no) AS won
        FROM matches_with_3_wickets mw
        JOIN bowling bo ON mw.t20_match_no = bo.t20_match_no
        JOIN match_result m ON bo.match_result_id = m.match_result_id
        WHERE m.match_result_id = 1  -- 1 indicates that the match was won
    )
    SELECT 
        (won_matches.won / total_matches.total) * 100 AS win_percentage
    FROM 
        total_matches, won_matches;
    """
    result = execute_sql_query(test_sql_query)
    print("Query Result:", result)


(Decimal('88.2353'),)
Query Result: [(Decimal('88.2353'),)]
